In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

Number of records in dataset

In [ ]:
df.count()

In [ ]:
#drop nulls
df = df.dropna()

df.count()

check datatypes of dataset to fit tables in schema



In [ ]:
#print schema
df.printSchema()

creating review_id table

In [ ]:
#create review_df to match review_id table

review_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])

review_df.show(5)

In [ ]:
#print schema

review_df.printSchema()

creating products table

In [ ]:
#create product dataframe to match products table(only unique values)
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

In [ ]:
#print schema

products_df.printSchema()

creating customers table

In [ ]:
#create customer dataframe to match customers table
from pyspark.sql.functions import desc
from pyspark.sql.types import TimestampType, IntegerType

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.show()

In [ ]:
#print schema

customers_df.printSchema()

creating vine table

In [ ]:
#create vine dataframe to match vine_table table

#load in a sql function to use columns
from pyspark.sql.functions import col

vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

#filter for only columns from Amazon's Vine program
#vine_df = vine_df.filter(col("vine")  == "Y")
#vine_df.select("vine").distinct().show()
#no reviews part of Vine program

vine_df.show() 

In [ ]:
#print schema

vine_df.printSchema()

load dataframe to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/video-game-db"
config = {"user":"<username>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

write to data

In [ ]:
#write dataframe to review_id_table table in RDS

review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
#write df to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
#write df to customer table in RDS


customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
#write df to vine_table table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)